# Run full epistasis pipeline

**Step 1:** Run the whole dataframe. All double-variant IDs are processed; embeddings are stored under `{output_base}/{source}/{model_key}.db` (split by the `source` column). Null is processed first so null covariance is saved and used for Mahalanobis metrics on non-null sources.

**Step 2:** Compute cov_inv from a subset (e.g. null only). Pass a subset dataframe; `compute_cov_inv` uses its sources and, if present, its epistasis_ids to fit covariance per model. Returns `{model_key: (cov, cov_inv)}`.

**Step 3:** Recompute metrics with that cov_inv and get a **series of tables**: one table per model (tool). Each table has the same structure (epistasis_id, source, len_WT_M1, epi_R_raw, epi_mahal, …); only the tool differs. No re-embedding; metrics are recomputed from the new DBs. Save each table separately (e.g. one parquet per tool) for downstream use.

Set paths and options in the config cell, then run Step 1, Step 2, and Step 3.

**Environment control:** Different models need different conda/envs (e.g. AlphaGenome needs JAX, Evo2 its own stack). Set **ENV_PROFILE** in the config cell and run this notebook in the matching environment:
- `"alphagenome"` — run only AlphaGenome (use AlphaGenome env)
- `"evo2"` — run only Evo2 (use Evo2 env)
- `"main"` — run all other models (shared env; excludes alphagenome & evo2)
- `"all"` — run every model (only if all deps in one env)

In [ ]:
# ---------------------------------------------------------------------------
# Config: paths and options
# ---------------------------------------------------------------------------
# Run with project root as current working directory (so "from notebooks.processing.process_epistasis" works).
# All data (embeddings, CSVs) lives under EPISTASIS_PAPER_ROOT. Override with env EPISTASIS_PAPER_ROOT.
import sys
from pathlib import Path

# Ensure repo root is on path so "notebooks" can be imported
ROOT = Path.cwd()
for _ in range(4):
    if (ROOT / "notebooks" / "paper_data_config.py").exists():
        break
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from notebooks.processing.process_epistasis import get_model_keys_for_env
from notebooks.paper_data_config import EPISTASIS_PAPER_ROOT, data_dir, embeddings_dir

# Environment profile: which models to run in *this* environment.
#   "alphagenome" -> only AlphaGenome (use in AlphaGenome conda env)
#   "evo2"       -> only Evo2 (use in Evo2 conda env)
#   "main"       -> all models except alphagenome & evo2 (shared env)
#   "all"        -> every model (only if all deps in one env)
ENV_PROFILE = "main"  # <-- single point of control: change per environment
MODEL_KEYS = get_model_keys_for_env(ENV_PROFILE)
print(f"ENV_PROFILE={ENV_PROFILE!r} -> MODEL_KEYS={MODEL_KEYS}")
print(f"Data root: {EPISTASIS_PAPER_ROOT}")

# Input mode: single dataframe with 'source' column, or list of (source_name, path) per source.
# - If USE_SINGLE_DATAFRAME is True: load one table (e.g. epistasis_aggregated.csv) with columns
#   epistasis_id and SOURCE_COL; storage dirs are split by value in SOURCE_COL: {output_base}/{source}/{model}.db.
# - If False: use SOURCES list below (one CSV per source).
USE_SINGLE_DATAFRAME = False
SINGLE_DATAFRAME_PATH = data_dir() / "epistasis_aggregated.csv"  # used when USE_SINGLE_DATAFRAME is True
# Resolve relative paths (e.g. "all_pairs_combined.csv") against data_dir so they work regardless of cwd
if USE_SINGLE_DATAFRAME and SINGLE_DATAFRAME_PATH is not None:
    p = Path(SINGLE_DATAFRAME_PATH)
    if not p.is_absolute() and not p.exists():
        p = data_dir() / p
        if p.exists():
            SINGLE_DATAFRAME_PATH = p
SOURCE_COL = "source"  # column in the single dataframe that defines the storage subdirectory per row

# (source_name, path_to_csv). Used when USE_SINGLE_DATAFRAME is False. Paths under paper data root/data/
SOURCES = [
    ("null", data_dir() / "null_epistasis.csv"),
    ("fas_analysis", data_dir() / "fas_subset.csv"),
    ("mst1r_analysis", data_dir() / "mst1r_subset.csv"),
    ("kras", data_dir() / "kras_subset.csv"),
    # ("tcga_analysis", data_dir() / "tcga_subset_doubles.csv"),
    # ("okgp_analysis", data_dir() / "okgp_subset_clean.csv"),
]

OUTPUT_BASE = embeddings_dir()
ID_COL = "epistasis_id"
BATCH_SIZE = 8  # add_epistasis_metrics batch_size (e.g. 8 => 32 sequences per batch)

# Optional: which models run on which sources. None = use MODEL_KEYS for all, with SpliceAI only on splicing sources.
# Example: {"null": ["nt500_multi", "convnova"], "fas_analysis": ["nt500_multi", "convnova", "spliceai"]}
SOURCE_MODEL_MAP = None

# Optional: list of directories to search for existing embeddings. For each epistasis_id, copy into new DB if found.
# If EMBEDDING_LOOKUP_FLAT is False: each base has base/source_name/model_key.db (same as OUTPUT_BASE).
# If True: each base has .db files directly (base/model_key.db), e.g. /path/embeddings/nt500_multi.db.
# Example: [Path("/path1/embeddings"), Path("/path2/embeddings")]
EMBEDDING_LOOKUP_BASES = None
EMBEDDING_LOOKUP_FLAT = True  # set True when each lookup dir has alphagenome.db, convnova.db, ... directly

# OpenSpliceAI checkpoint dir for SpliceAI (or set env OPENSPLICEAI_MODEL_DIR)
SPLICEAI_MODEL_DIR = None  # e.g. "/path/to/openspliceai-mane/10000nt"

In [ ]:
# ---------------------------------------------------------------------------
# Step 1: Process all sources (null first); one .db per model per source
# ---------------------------------------------------------------------------
# Either run from a single dataframe (storage split by SOURCE_COL) or from SOURCES list.
import logging
import pandas as pd
from notebooks.processing.process_epistasis import run_sources, run_from_single_dataframe

logging.basicConfig(level=logging.INFO)

if USE_SINGLE_DATAFRAME and SINGLE_DATAFRAME_PATH is not None and Path(SINGLE_DATAFRAME_PATH).exists():
    df_all = pd.read_csv(SINGLE_DATAFRAME_PATH)
    run_from_single_dataframe(
        df_all,
        output_base=OUTPUT_BASE,
        source_col=SOURCE_COL,
        model_keys=MODEL_KEYS,
        source_model_map=SOURCE_MODEL_MAP,
        embedding_lookup_bases=EMBEDDING_LOOKUP_BASES,
        embedding_lookup_flat=EMBEDDING_LOOKUP_FLAT,
        spliceai_model_dir=SPLICEAI_MODEL_DIR,
        id_col=ID_COL,
        show_progress=True,
        force=False,
        batch_size=BATCH_SIZE,
    )
    # For Step 2: source names in same order (null first)
    _unique = df_all[SOURCE_COL].dropna().astype(str).unique().tolist()
    SOURCE_NAMES = [s for s in _unique if s == "null"] + [s for s in _unique if s != "null"]
else:
    df_all = None  # so Step 2 can check and use a small df for source subset
    run_sources(
        SOURCES,
        output_base=OUTPUT_BASE,
        model_keys=MODEL_KEYS,
        source_model_map=SOURCE_MODEL_MAP,
        embedding_lookup_bases=EMBEDDING_LOOKUP_BASES,
        embedding_lookup_flat=EMBEDDING_LOOKUP_FLAT,
        spliceai_model_dir=SPLICEAI_MODEL_DIR,
        id_col=ID_COL,
        show_progress=True,
        force=False,
        batch_size=BATCH_SIZE,
    )
    SOURCE_NAMES = [name for name, _ in SOURCES]

In [ ]:
# ---------------------------------------------------------------------------
# Step 2: Compute cov_inv from a part of the dataframe — returns (cov, cov_inv) directly
# ---------------------------------------------------------------------------
# Pass a subset of your data (by source). You get back a dict: {model_key: (cov, cov_inv)}.
from notebooks.processing.process_epistasis import compute_cov_inv

# Define which sources to use (e.g. null only, or null + fas_analysis).
# If you ran Step 1 from a single dataframe, filter it; otherwise use a small df with source names.
try:
    _df = df_all
except NameError:
    _df = None
if _df is not None:
    df_subset = _df[_df[SOURCE_COL].isin(["null"])]   # e.g. null only
else:
    df_subset = __import__("pandas").DataFrame({SOURCE_COL: ["null"]})  # null only by source name

cov_inv_by_model = compute_cov_inv(
    OUTPUT_BASE,
    source_df=df_subset,
    source_col=SOURCE_COL,
    id_col=ID_COL,
    model_keys=MODEL_KEYS,
    method="ledoit_wolf",
    show_progress=True,
)
# Example: get arrays for one model
# cov, cov_inv = cov_inv_by_model["nt500_multi"]
# cov.shape, cov_inv.shape
print("Returned cov_inv for models:", list(cov_inv_by_model.keys()))
for k, (c, ci) in cov_inv_by_model.items():
    print(f"  {k}: cov {c.shape}, cov_inv {ci.shape}")

In [ ]:
# ---------------------------------------------------------------------------
# Step 3: Recompute metrics with cov_inv → one table per tool (same structure each)
# ---------------------------------------------------------------------------
# Uses embeddings already in OUTPUT_BASE; only metric columns are recomputed.
# Returns a dict: {model_key: DataFrame}. Each table has the same columns
# (epistasis_id, source, len_WT_M1, epi_R_raw, epi_mahal, ...); one table per tool.
from notebooks.processing.process_epistasis import recompute_metrics_with_cov_inv

# Full dataframe (must have epistasis_id and source). Use df_all from Step 1 or load your table.
try:
    _df_full = df_all
except NameError:
    _df_full = None
if _df_full is None and USE_SINGLE_DATAFRAME and SINGLE_DATAFRAME_PATH is not None and Path(SINGLE_DATAFRAME_PATH).exists():
    _df_full = pd.read_csv(SINGLE_DATAFRAME_PATH)
if _df_full is None or len(_df_full) == 0:
    raise ValueError("Need full dataframe with epistasis_id and source. Run Step 1 first or set SINGLE_DATAFRAME_PATH.")

metrics_by_tool = recompute_metrics_with_cov_inv(
    OUTPUT_BASE,
    _df_full,
    cov_inv_by_model,
    source_col=SOURCE_COL,
    model_keys=list(cov_inv_by_model),
    id_col=ID_COL,
    spliceai_model_dir=SPLICEAI_MODEL_DIR,
    batch_size=BATCH_SIZE,
    show_progress=True,
)
# One table per model; same structure (epistasis_id, source, len_WT_M1, epi_mahal, ...)
print("Series of tables: one per tool, same columns in each")
for tool, tbl in metrics_by_tool.items():
    print(f"  {tool}: {tbl.shape[0]} rows, columns {list(tbl.columns)[:6]}...")

# Optional: save one table per tool (e.g. epistasis_metrics_nt500_multi.parquet)
# for tool, tbl in metrics_by_tool.items():
#     tbl.to_parquet(data_dir() / f"epistasis_metrics_{tool}.parquet", index=False)

In [ ]:
# Unpack for one model (e.g. for downstream use or add_epistasis_metrics with cov_inv=...)
# cov, cov_inv = cov_inv_by_model["nt500_multi"]

In [ ]:
# Optional: save cov_inv to .npz if you need it on disk later
# from notebooks.processing.process_epistasis import run_covariance_and_save
# run_covariance_and_save(OUTPUT_BASE, source_names, model_keys=MODEL_KEYS, out_npz_dir=OUTPUT_BASE)